In [ ]:
# Solution Ex. 50_new

In [ ]:
inputPath  = "/data/students/bigdata-01QYD/ex_data/Ex50_new/data/sensors.txt"
outputPath = "res_out_Ex50_newSQL/"

In [ ]:
# Create a DataFrame from persons.csv
readingDF = spark.read.load(inputPath,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "sensorId")\
.withColumnRenamed("_c1", "date")\
.withColumnRenamed("_c2", "PM10")

In [ ]:
readingDF.printSchema()
readingDF.show()

In [ ]:
# "register" readingDF
readingDF.createOrReplaceTempView("readings")

In [ ]:
# Register a UDF that returns:
# +1 if PM10>50
# -1 otherwise
spark.udf.register("Critical", lambda PM10: +1 if (PM10>50) else -1)

In [ ]:
# Define one group for each value of sensorId and "count" the number of records with 
# PM10>50 with respect to those with PM10<=50.
# If the sum is greater than 0, the sensor must be selected because the number of 
# critical dates is greater than the number of normal dates 
selectedSensors = \
spark.sql("""SELECT sensorId
FROM readings
GROUP BY sensorId
HAVING SUM(Critical(PM10))>0""")

#spark.sql("""SELECT sensorId, SUM(Critical(PM10)) 

In [ ]:
# Debug
selectedSensors.printSchema()
selectedSensors.show()

In [ ]:
# Store the result in the output folder
selectedSensors\
.write.csv(outputPath, header=False)

In [ ]:
#!hdfs dfs -rm -r res_out_Ex50_newSQL/